In [587]:
import csv

import gurobipy as gp
from gurobipy import GRB

In [588]:

# _____________________________________ Constants _____________________________________

STATE = "MS"
THRESHOLD_PCT = 0.1

# _____________________________________ Load the data _____________________________________

# Distances between nodes matrix
distances = []
with open(f"data/{STATE}/{STATE}_distances.csv", "r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader, None)  # Skip the header
    distances = [list(map(int, row[1:])) for row in reader]  # Read line skipping NodeID
print(f"Distances:\n{distances}")

# Number of nodes
nodes_count = len(distances)
print(f"Nodes count: {nodes_count}")

# Number of districts
districts_count = 0

# Adjacency matrix
adjacency_matrix = [[0 for _ in range(nodes_count)] for _ in range(nodes_count)]
edge_list = []
with open(f"data/{STATE}/{STATE}.dimacs", "r", encoding="utf-8") as file:
    for line in file:
        if line.startswith("e"):
            _, node1, node2 = line.split()
            node1 = int(node1)
            node2 = int(node2)
            adjacency_matrix[node1][node2] = 1
            adjacency_matrix[node2][node1] = 1
            edge_list.append((node1, node2))
        if line.startswith("c"):
            _, _, d_count = line.split()
            districts_count = int(d_count)

print(f"Adjacency matrix:\n{adjacency_matrix}")
print(f"Edge list: {edge_list}")

print(f"Districts count: {districts_count}")

# Population vector
population = []
total_population = 0
with open(f"data/{STATE}/{STATE}.population", "r", encoding="utf-8") as file:
    total_population = int(file.readline().split()[-1])
    for line in file:
        population.append(int(line.split()[1]))

print(f"Population vector: {population}")
print(f"Total population: {total_population}")

Distances:
[[0, 291, 150, 337, 53, 161, 345, 96, 89, 256, 98, 129, 138, 43, 134, 81, 169, 189, 188, 224, 304, 142, 157, 305, 347, 238, 366, 298, 175, 124, 236, 129, 131, 306, 239, 141, 172, 314, 110, 362, 36, 120, 75, 27, 221, 120, 59, 260, 321, 151, 225, 183, 355, 229, 48, 371, 302, 204, 86, 343, 206, 192, 353, 96, 74, 270, 147, 94, 263, 182, 57, 266, 93, 47, 102, 79, 51, 122, 270, 86, 173, 209], [291, 0, 204, 96, 313, 136, 65, 362, 293, 42, 336, 377, 427, 249, 305, 288, 126, 184, 131, 139, 144, 422, 419, 85, 73, 87, 93, 115, 401, 169, 67, 351, 163, 34, 159, 334, 373, 63, 254, 113, 289, 205, 356, 304, 72, 199, 245, 35, 30, 167, 205, 245, 67, 100, 338, 84, 44, 119, 327, 132, 97, 99, 61, 387, 339, 177, 267, 203, 109, 222, 337, 66, 244, 269, 381, 233, 340, 375, 63, 207, 157, 90], [150, 204, 0, 212, 139, 83, 268, 178, 101, 162, 240, 182, 279, 119, 102, 204, 132, 41, 175, 86, 162, 290, 307, 180, 235, 123, 295, 163, 199, 82, 179, 152, 120, 231, 93, 131, 170, 252, 57, 235, 173, 168, 183, 145

In [589]:
m = nodes_count  # Number of territories
n = districts_count  # Number of districts
d = distances  # Distance matrix (m x m)
p = population  # Population vector of length m
A = adjacency_matrix  # Adjacency matrix (m x m)
a = (total_population / districts_count) - (
    total_population / districts_count
) * THRESHOLD_PCT/2  # Lower bound constraint on population of districts
b = (total_population / districts_count) + (
    total_population / districts_count
) * THRESHOLD_PCT/2  # Upper bound constraint on population of districts

import math
L = math.ceil((1-THRESHOLD_PCT/2)*total_population/districts_count)
U = math.floor((1+THRESHOLD_PCT/2)*total_population/districts_count)

L, U

(703304, 777335)

In [590]:
print("Using L =",a,"and U =",b,"and k =",n)

Using L = 703303.7625 and U = 777335.7375 and k = 4


In [591]:
model = gp.Model()

# create x[i,j] variable which equals one when county i 
#    is assigned to (the district centered at) county j
x = model.addVars(m, m, vtype=GRB.BINARY, name="x")


In [592]:
# objective is to minimize the moment of inertia: d^2 * p * x
model.setObjective(
    gp.quicksum(
        d[i][j] * d[i][j] * p[i] * x[i, j] for i in range(m) for j in range(m)
    ),
    GRB.MINIMIZE,
)

In [593]:
# add constraints saying that each county i is assigned to one district
model.addConstrs( gp.quicksum(x[i,j] for j in range(m)) == 1 for i in range(m))

# add constraint saying there should be k district centers
model.addConstr( gp.quicksum( x[j,j] for j in range(m) ) == n )

# add constraints that say: if j roots a district, then its population is between L and U.
model.addConstrs( gp.quicksum( p[i] * x[i,j] for i in range(m)) >= a * x[j,j] for j in range(m) )
model.addConstrs( gp.quicksum( p[i] * x[i,j] for i in range(m)) <= b * x[j,j] for j in range(m) )

# add coupling constraints saying that if i is assigned to j, then j is a center.
model.addConstrs( x[i,j] <= x[j,j] for i in range(m) for j in range(m))

model.update()

In [594]:
# Add contiguity constraints
%pip install gerrychain
%pip install geopandas
from gerrychain import Graph

G = Graph(edge_list)
G

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


<Graph [82 nodes, 202 edges]>

In [595]:
for node in G.nodes:
    G.nodes[node]['population'] = population[node]

G.nodes[0]['population']


18340

In [596]:

import networkx as nx
DG = nx.DiGraph(G)

# Add variable f[j,u,v] which equals the amount of flow (originally from j) that is sent across arc (u,v)
f = model.addVars( DG.nodes, DG.edges, vtype=GRB.CONTINUOUS)
M = DG.number_of_nodes()-1

# Add constraint saying that node j cannot receive flow of its own type
model.addConstrs( gp.quicksum( f[j,u,j] for u in DG.neighbors(j)) == 0 for j in DG.nodes )

# Add constraints saying that node i can receive flow of type j only if i is assigned to j
model.addConstrs( gp.quicksum( f[j,u,i] for u in DG.neighbors(i)) <= M * x[i,j] for i in DG.nodes for j in DG.nodes if i != j )

# If i is assigned to j, then i should consume one unit of j flow. 
#    Otherwise, it should consume no units of j flow.
model.addConstrs( gp.quicksum( f[j,u,i] - f[j,i,u] for u in DG.neighbors(i)) == x[i,j] for i in DG.nodes for j in DG.nodes if i != j )

model.update()

DG

In [597]:
# Print number of variables and constraints
print("Number of variables: ", model.NumVars)
print("Number of constraints: ", model.NumConstrs)


Number of variables:  39852
Number of constraints:  20337


In [598]:
# solve, making sure to set a 0.00% MIP gap tolerance(!)
model.Params.MIPGap = 0.0
model.optimize()

Set parameter MIPGap to value 0
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20337 rows, 39852 columns and 145398 nonzeros
Model fingerprint: 0x26f16931
Variable types: 33128 continuous, 6724 integer (6724 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+05]
  Objective range  [7e+05, 5e+10]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 164 rows and 404 columns
Presolve time: 0.26s
Presolved: 20173 rows, 39448 columns, 144588 nonzeros
Variable types: 32724 continuous, 6724 integer (6724 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 1.56

In [599]:
print("The moment of inertia objective is",model.objval)

# retrieve the districts and their populations
centers = [j for j in G.nodes if x[j,j].x > 0.5 ]
districts = [ [i for i in G.nodes if x[i,j].x > 0.5] for j in centers]
district_counties = [ [ G.nodes[i] for i in districts[j] ] for j in range(districts_count)]
district_populations = [ sum(G.nodes[i]["population"] for i in districts[j]) for j in range(districts_count) ]

# print district info
for j in range(districts_count):
    print("District",j,"has population",district_populations[j],"and contains counties",district_counties[j])

The moment of inertia objective is 15623364045.0
District 0 has population 759102 and contains counties [{'population': 31184}, {'population': 13266}, {'population': 83343}, {'population': 17106}, {'population': 27777}, {'population': 55813}, {'population': 28064}, {'population': 33208}, {'population': 33752}, {'population': 185314}, {'population': 9782}, {'population': 25008}, {'population': 18850}, {'population': 23863}, {'population': 34740}, {'population': 21815}, {'population': 21629}, {'population': 12481}, {'population': 12715}, {'population': 21390}, {'population': 6176}, {'population': 7646}, {'population': 34180}]
District 1 has population 708866 and contains counties [{'population': 18340}, {'population': 14209}, {'population': 11321}, {'population': 25949}, {'population': 227742}, {'population': 1338}, {'population': 44722}, {'population': 3800}, {'population': 12016}, {'population': 28368}, {'population': 13884}, {'population': 34907}, {'population': 40324}, {'population':